In [196]:
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder

In [197]:
df=pd.read_csv(r'C:\Users\pavan\Desktop\data science\logistic regression\research_grid(hyperparameter_tuning)\loan_approval_dataset.csv')

In [198]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Age                1000 non-null   int64 
 1   Salary             1000 non-null   int64 
 2   Credit_Score       1000 non-null   int64 
 3   Loan_Amount        1000 non-null   int64 
 4   Loan_Term          1000 non-null   object
 5   Employment_Status  1000 non-null   object
 6   Residence_Type     1000 non-null   object
 7   Previous_Default   1000 non-null   object
 8   Loan_Approved      1000 non-null   object
dtypes: int64(4), object(5)
memory usage: 70.4+ KB


In [199]:
cat_col=['Loan_Term','Employment_Status','Residence_Type','Previous_Default']
num_cols=df.select_dtypes(include='number').columns

In [200]:
num_pipelines=Pipeline(steps=[
('scaling-for_num_cols',StandardScaler())
])

In [201]:
cat_pipelines=Pipeline(steps=[
('encoding_for_cat_cols',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1))
])


In [202]:
preprocessing=ColumnTransformer(
    transformers=[
        ('num_pipelines',num_pipelines,num_cols),
        ('cat_pipelines',cat_pipelines,cat_col)
    ]
)


In [203]:
main_pipeline=Pipeline(
    steps=[
        ('preprocessing',preprocessing),
        ('model',LogisticRegression(max_iter=1000))
    ]
)

In [204]:
x=df.drop(columns='Loan_Approved')
y=df['Loan_Approved']

In [205]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.8,random_state=42)

In [206]:
grid=GridSearchCV(

    estimator=main_pipeline,
    param_grid={
        'model__C':[0.01,0.1,1.0,10],
        'model__penalty':['l1','l2'],
        'model__solver':['liblinear'],
        'model__class_weight':['balanced',None]
    },
    cv=5,
    n_jobs=-1,
    verbose=0,
    scoring='f1_macro'
)

In [207]:
grid.fit(xtrain,ytrain)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num_pipelines',
                                                                         Pipeline(steps=[('scaling-for_num_cols',
                                                                                          StandardScaler())]),
                                                                         Index(['Age', 'Salary', 'Credit_Score', 'Loan_Amount'], dtype='object')),
                                                                        ('cat_pipelines',
                                                                         Pipeline(steps=[('encoding_for_cat_cols',
                                                                                          OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                         unknown_value=-1))]),
                                                                         ['Loan_Term',
                                                                          'Employment_Status',
                                                                          'Residence_Type',
                                                                          'Previous_Default'])])),
                                       ('model',
                                        LogisticRegression(max_iter=1000))]),
             n_jobs=-1,
             param_grid={'model__C': [0.01, 0.1, 1.0, 10],
                         'model__class_weight': ['balanced', None],
                         'model__penalty': ['l1', 'l2'],
                         'model__solver': ['liblinear']},
             scoring='f1_macro')

In [208]:
model=grid.best_estimator_

In [209]:
model.score(xtrain,ytrain)

0.54625

In [210]:
model.score(xtest,ytest)

0.52